In [21]:
import time_series_pred_model

import torch

data = torch.load("items.json")


In [22]:
import pandas as pd
import random
# sample_items = random.sample(list(data.items()), 10)
# for key, value in sample_items:
#     print(key, value)

df_tensor = pd.DataFrame.from_dict(data, orient='index')

# Функция для извлечения тензора из value словаря
def extract_tensor(x):
    if isinstance(x, dict):
        return pd.Series(x).apply(extract_tensor)
    elif isinstance(x, torch.Tensor):
        return x
    else:
        return None

# Применение функции к каждому элементу DataFrame и преобразование в Series
s = df_tensor.applymap(extract_tensor).stack()

print(s)

4D5CF815F5D9F81391510D421CF567EF  prices    [tensor(1000000.), tensor(1000000.), tensor(10...
                                  cnt       [tensor(0.), tensor(0.), tensor(0.), tensor(0....
1A1D8A4A6BD5E56958128A32B086A2C0  prices    [tensor(1000000.), tensor(1000000.), tensor(11...
                                  cnt       [tensor(0.), tensor(0.), tensor(1.), tensor(0....
3506A17DB897B0F35F01FBD6983B94F4  prices    [tensor(1000000.), tensor(1000000.), tensor(10...
                                                                  ...                        
BA0C00625E89155A74FEB264E3371410  cnt       [tensor(0.), tensor(0.), tensor(0.), tensor(0....
978FC8C0D9486076C4DB7CFBF568ED09  prices    [tensor(1000000.), tensor(1000000.), tensor(10...
                                  cnt       [tensor(0.), tensor(0.), tensor(0.), tensor(0....
8E6D567CB89FE0A58503EB17042E1920  prices    [tensor(73571.3359), tensor(41383.8750), tenso...
                                  cnt       [tensor(3.), ten

In [23]:
# получаем список тензоров для ключа "a"
tensor_list = [v["cnt"] for v in data.values() if isinstance(v["cnt"], torch.Tensor)]

# создаем серию из списка тензоров
tensor_series = pd.Series(tensor_list)

print(tensor_series)

0       [tensor(0.), tensor(0.), tensor(0.), tensor(0....
1       [tensor(0.), tensor(0.), tensor(1.), tensor(0....
2       [tensor(0.), tensor(0.), tensor(0.), tensor(0....
3       [tensor(8.), tensor(3.), tensor(3.), tensor(2....
4       [tensor(0.), tensor(0.), tensor(0.), tensor(0....
                              ...                        
9084    [tensor(0.), tensor(0.), tensor(0.), tensor(0....
9085    [tensor(0.), tensor(0.), tensor(0.), tensor(0....
9086    [tensor(0.), tensor(0.), tensor(0.), tensor(0....
9087    [tensor(0.), tensor(0.), tensor(0.), tensor(0....
9088    [tensor(3.), tensor(8.), tensor(11.), tensor(7...
Length: 9089, dtype: object


In [24]:
type(tensor_series[0])

most_popular_product_tensor = data["0F40E9DE2E7DBC0DA161448D98D47C69"]['cnt']
print(most_popular_product_tensor)

tensor([ 33.,  34.,  23.,  30.,  65.,  98.,  77.,  20.,  26.,  35.,  31.,  48.,
         68.,  72.,  35.,  35.,  36.,  39.,  56.,  95.,  72.,  30.,  33.,  89.,
         69., 104., 169., 119.,  70.,  70.,  68.,  96., 106., 150., 117., 116.,
         86., 120., 101.,  84.,  14.,  43.,  59.,  47.,  40.,  51.,  35.,  32.,
         54.,  29.,  46.,  32.,  53.,  40.,  68.,  68.,  37.,  36.,  58.,  39.,
         45.,  82.,  64.,  30.,  53.,  48.,  46.,  42.,  81.,  70.,  35.,  30.,
         32.,  31.,  39.,  60.,  55.,  32.,  44.,  37.,  46.,  54.,  83.,  84.,
         59.,  45.,  36.,  44.,  70., 108., 112.,  44.,  51.,  75.,  42.,  54.,
         73.,  66.,  40.,  75.,  62.,  58.,  77., 114., 174., 168., 151.,  83.,
         57.,  64., 124., 103.,  46.,  60.,  52.,  52.,  64.,  72.,  90.,  69.,
         54.,  69.,  63.,  75.,  87.,  73.,  45.,  59.,  58.,  63.,  59.,  82.,
         77.,  48.,  45.,  51.,  68.,  61., 117., 113.,  45.,  42.,  67.,  43.,
         74., 122.,  98.,  60.,  49.,  4

In [27]:
def split_tensor_for_time_series(tensor):
    tensor_split = torch.split(tensor, int(len(tensor)*0.8), dim=0)
    return pd.Series(tensor_split[0]), pd.Series(tensor_split[1])


time_series_model = time_series_pred_model.Time_series_model(4, 73, 1) # 4 периода по 73 дня.
# Предсказывает следующий период длины 73. Можно поменять и сделать периодов больше длины 5 дней
# И тогда будет предсказывать следующие 5 дней. Главное правильной длины тензоры закидывайте и всё норм будет.

train, test = split_tensor_for_time_series(tensor_series[6])


time_series_model.fit(train)


In [28]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

test_pred = time_series_model.predict(test)
mae = round(mean_absolute_error(test, test_pred), 2)
mape = round(mean_absolute_percentage_error(test, test_pred), 3)
print("MAE:", mae)
print("MAPE:", mape)

MAE: 0.47
MAPE: 2049825507533163.5
